In [67]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from datetime import datetime as dt

In [68]:
tr=pd.read_csv('../dataset/updated_train.csv')
te=pd.read_csv('../dataset/updated_test.csv')
date=pd.read_csv('../input/date_info.csv')

In [69]:
#1,2列目を削除して、Nanを0で埋める
def clean(df):
    buf=df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)
    df=buf.fillna(0)
    return df

In [70]:
#不要な列を削除し、列名を修正する
def deletecol(df):
    delcol=['reserve_datetime_diff_x',
            'reserve_visitors_x',
            'reserve_datetime_diff_y',
            'reserve_visitors_y',
            'air_genre_name_y',
            'air_area_name_y',
            'latitude_y',
            'longitude_y',
            'latitude_str',
            'longitude_str',
            'station_id',
            'station_latitude',
            'station_longitude',
            'station_vincenty',
            'station_great_circle'
           ]
    buf=df.drop(delcol,axis=1)
    df=buf.rename(columns={'air_genre_name_x':'air_genre_name',
                                                'air_area_name_x':'air_area_name',
                                                'latitude_x': 'latitude',
                                                'longitude_x':'longitude'})
    return df

In [71]:
#holiday_flgをmergeする
def holiday(tr,date):
    buf_date=date.drop('day_of_week',axis=1)
    tr=pd.merge(tr,buf_date,on='calendar_date',how='left')
    return tr

In [72]:
#idをid_dateの形に変形
def id_date(df):
    buf=df.copy()
    buf['store_id_date']=buf['air_store_id']+'_'+buf['calendar_date']
    buf=buf.drop(['air_store_id','calendar_date'],axis=1)
    buf=buf.set_index('store_id_date')
    return buf

In [73]:
#holiday_flgにゴールデンウィークのflgを設定する
def golden(date):
    golden_flg=['2016-04-30',
                        '2016-05-01',
                        '2016-05-02'
                            ]
    
    for flg in golden_flg:
        for i,d in enumerate(date['calendar_date']):
            if d == flg:
                date.iloc[[i],[2]]=1
    return date

In [74]:
def mod_calendar(df):
    ymd=[]
    for i in te2['calendar_date']:
        m,d,y=i.split('/')
        if len(d)==1:
            d='0'+d
        ymd.append(y+'-0'+m+'-'+d)
    df['calendar_date']=ymd
    return df

In [75]:
def index(df):
    buf=df.copy()
    buf=buf.drop(['air_store_id','calendar_date'],axis=1)
    buf=buf.set_index('id')
    return buf

In [76]:
tr1=clean(tr)
tr2=deletecol(tr1)
date=golden(date)
tr3=holiday(tr2,date)
tr4=id_date(tr3)

In [77]:
tr4.to_csv('../dataset/updated_train1.csv')

In [78]:
te1=clean(te)
te2=deletecol(te1)
te3=mod_calendar(te2)
te4=holiday(te2,date)
te5=index(te4)

In [79]:
te5.to_csv('../dataset/updated_test1.csv')